Before we do any work, we need to import several functions from cdapython:
- `Q` and `query` which power the search
- `columns` which lets us view entity field names
- `unique_terms` which lets view entity field contents

We're also asking cdapython to report it's version so we can be sure we're using the one we mean to.

In [1]:
from cdapython import Q, columns, unique_terms, query
import cdapython
print(cdapython.__version__)
Q.set_host_url("http://35.192.60.10:8080/")

2022.4.13


Note that I'm working off a testing version, these are new features that will come out with our next release, so if you go try these things in the current stable version they won't work. When we do release, this notebook will be available for you to run and play with.

CDA data comes from three sources:
- The [Proteomic Data Commons](https://proteomic.datacommons.cancer.gov/pdc/) (PDC)
- The [Genomic Data Commons](https://gdc.cancer.gov/) (GDC)
- The [Imaging Data Commons](https://datacommons.cancer.gov/repository/imaging-data-commons) (IDC)

The CDA makes this data searchable in two ways, by a "Subject" table and by "File" table. 

`Subject` data is information that is intrisic to the individual under study, e.g. `sex`, `race`, `ethnicity`. However any given subject might be part of multiple studies. To make search across datasets easier, the CDA model aggregates this data as `ResearchSubject` information. Subjects that participate in multiple projects (are part of multiple nodes), will have multiple `ResearchSubject` entries.

`Subject` and `ResearchSubject` fields are available for both the "Subject" and "File" tables, however terms specific to files e.g. 'data_type' or 'file_format' are only available in "File" table.

To see what fields are available, we use the command `columns`. Here we're looking at the first ten metadata fields available for files:

In [2]:
columns(files=True, limit=10)

['id',
 'identifier',
 'identifier.system',
 'identifier.value',
 'label',
 'data_category',
 'data_type',
 'file_format',
 'associated_project',
 'drs_uri']

By default, `columns()` returns `Subject` table fields. The first several fields (those without a `.` in them) are `Subject` demographic information, which is intrinsically attached to a given subject. Subsequent entities (i.e. `Research.Subject.xxx`) contain details about specific experiments the subject was part of. They are equivilent to the nodes' `Case` record in the GDC and PDC. 

While available search fields may look like ones you've seen in PDC, GDC or IDC, that does not mean they will contain exactly the same information; several are renamed or restructured in the CDA model. The field name mappings are described in [CDA Schema Field Mapping](../Documentation/Schema.md), but we can also directly get information about what data populates any of these fields using the `unique_terms()` function:

In [3]:
unique_terms("ResearchSubject.primary_diagnosis_site", files=True, limit=10)

[None,
 'Adrenal gland',
 'Anus and anal canal',
 'Base of tongue',
 'Bladder',
 'Bones, joints and articular cartilage of limbs',
 'Bones, joints and articular cartilage of other and unspecified sites',
 'Brain',
 'Breast',
 'Bronchus and lung']

The CDA provides a custom python tool for searching CDA data. [`Q`](usage/#q) (short for Query) offers several ways to search and filter data, and several input modes:

---

- **[Q.run()](../../../Documentation/usage/#qrun)** returns **demographic** data for the specified search 
- **[Q.counts()](../../../Documentation/usage/#qcounts)** returns summary information (counts) for the **files** that fit the specified search
- **[Q.files()](../../../Documentation/usage/#qfiles)** returns data for the **files** that fit the specified search
- **[Q.sql()](../../../Documentation/usage/#qsql)** allows you to use SQL syntax instead of Q syntax 
- **[query()](../../../Documentation/usage/#query)** allows you to use a more natural language syntax of Q

---


For todays demo, I'm going to show you how you can use `Q.counts()`, `Q.files()`, and `Q.run()` to build a cohort

## Retrieving summary information

### Demographic summary
Let's run a search with a relatively simple question: We're interested in finding data about Kidney cancer. To run this simple search, we would first construct a query in `Q` and save it to a variable `myquery`:

In [4]:
myquery = Q('ResearchSubject.primary_diagnosis_site = "Kidney"')

Since we are looking for demographic summary information, we want to use this query in Q.run. We do this by running .run() on the query we just saved, and saving the result to a new variable mydemographic:

In [5]:
mydemographic = myquery.run(version="all_Files_v3_0_w_RS")

Getting results from database

Total execution time: 5054 ms


We're saving information in variables, so we don't get any visible output. To see what our results are, we need to look into the variable. The simplest way is to call `mydemographic` directly:

In [6]:
mydemographic


            QueryID: 83cab934-745e-417f-9a97-3c93389ab89d
            Query:SELECT all_Files_v3_0_w_RS.* FROM gdc-bq-sample.dev.all_Files_v3_0_w_RS AS all_Files_v3_0_w_RS, UNNEST(all_Files_v3_0_w_RS.ResearchSubject) AS _ResearchSubject WHERE (UPPER(_ResearchSubject.primary_diagnosis_site) = UPPER('Kidney'))
            Offset: 0
            Count: 100
            Total Row Count: 506820
            More pages: True
            

This output tells us our QueryID, which we don't really need, but the computer does to track our questions. 
Then it tells us five parameters that describe our results:

---

- **Query:** This is the actual SQL query that was run on our database to retreive your results
- **Offset:** This is how many rows of information we've told the query to skip in the data, here we didn't tell it to skip anything, so the offset is zero
- **Count:** This is how many rows (Subjects) the current page of our results table has. To keep searches fast, we default to pages with 100 rows.
- **Total Row Count:** This is how many rows (Subjects) are in the full results table
- **More pages:** This is alwasys a True or False. False means that our current page has all the availble results. True means that we will see only the first 100 results in this table, and will need to page through for more.

---
    
Now that we've seen the metadata about our results, let's look at the actual table. The easiest way to do this is by using the python function `.to_dataframe()` on our `mydemographic` variable:

In [7]:
mydemographic.to_dataframe()

,id,identifier,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number,Subject,ResearchSubject,Specimen
0,f3e8ea5a-cd19-4d0b-9884-81f1f680fabb,"[{'system': 'GDC', 'value': 'f3e8ea5a-cd19-4d0...",7f61f0f6-bf10-4119-96ba-7a92017bad6b.rna_seq.t...,Sequencing Reads,Aligned Reads,BAM,TCGA-KIRC,drs://dg.4DFC:f3e8ea5a-cd19-4d0b-9884-81f1f680...,14359653313,280c1164b6d48b5ee8a704bc6a4d2974,Genomic,None,None,"[{'id': 'TCGA-CJ-4634', 'identifier': [{'syste...",[{'id': '852effcd-8146-46e1-96fa-e5a9f09a88e2'...,[{'id': 'c677da58-ed7b-43ae-b49b-7a7046f828b1'...
1,66b1f91a-937c-4e1f-b4ab-07d1ef9fe1db,"[{'system': 'GDC', 'value': '66b1f91a-937c-4e1...",94b39932-9777-4fdc-b56c-65c3c3e11053.rna_seq.a...,Transcriptome Profiling,Gene Expression Quantification,TSV,TCGA-KIRC,drs://dg.4DFC:66b1f91a-937c-4e1f-b4ab-07d1ef9f...,4243348,6222c0a06514a48a6afac961b365bfd9,Genomic,None,None,"[{'id': 'TCGA-B0-4714', 'identifier': [{'syste...",[{'id': '98ff8a6b-0ead-4e55-9fb7-6fe5cc69d0f6'...,[{'id': '37ebcaeb-7a9c-4e96-adf2-49aa7e402afd'...
2,5ae9c6bb-86a9-4654-946a-0b23aff00ca7,"[{'system': 'GDC', 'value': '5ae9c6bb-86a9-465...",TCGA-KIRC.f885f6d1-3b9b-40f8-a48b-f0fa0e5791d7...,Structural Variation,Transcript Fusion,TSV,TCGA-KIRC,drs://dg.4DFC:5ae9c6bb-86a9-4654-946a-0b23aff0...,10217,b3e901691f26fb972a5884a688dda397,Genomic,None,None,"[{'id': 'TCGA-CJ-5684', 'identifier': [{'syste...",[{'id': '9f4cd3b9-50ef-4f85-84b8-acb0b77887d1'...,[{'id': '6ce66cbc-9dcf-48f6-a70a-ee50523634b0'...
3,a8ba8a6a-aa56-4602-8b79-7a02c22c1ffd,"[{'system': 'GDC', 'value': 'a8ba8a6a-aa56-460...",d96c15e3-28ba-448c-bee7-1ed38ccabbec.rna_seq.c...,Sequencing Reads,Aligned Reads,BAM,TCGA-KIRC,drs://dg.4DFC:a8ba8a6a-aa56-4602-8b79-7a02c22c...,86192678,8fa19823d70338759f2f3372812f285d,Genomic,None,None,"[{'id': 'TCGA-CZ-5452', 'identifier': [{'syste...",[{'id': 'e9faa588-d45a-450a-81a2-949eb2834bc0'...,[{'id': 'de8225ab-7150-4a1f-a2b2-5cf2ba1137b9'...
4,f5c19f4d-7fb6-485c-8a2a-2df03ef098c6,"[{'system': 'GDC', 'value': 'f5c19f4d-7fb6-485...",8904adeb-fde3-4779-aca1-0a038a622cde.wxs.MuTec...,Simple Nucleotide Variation,Annotated Somatic Mutation,MAF,TCGA-KIRC,drs://dg.4DFC:f5c19f4d-7fb6-485c-8a2a-2df03ef0...,1793177,3b01feb24759faf3127519ab18ea04e0,Genomic,None,None,"[{'id': 'TCGA-A3-3363', 'identifier': [{'syste...",[{'id': '659294b9-ded9-498a-bc00-c1d4d456aa4a'...,[{'id': '605e86ac-58f6-47d4-8bde-15eb23c857cc'...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,53a272d2-958d-4473-8345-d22528c227af,"[{'system': 'IDC', 'value': '53a272d2-958d-447...",tasets-idc/53a272d2-958d-4473-8345-d22528c227a...,Imaging,None,DICOM,c4kc_kits,drs://dg.4DFC:53a272d2-958d-4473-8345-d22528c2...,None,None,Imaging,CT,None,"[{'id': 'KiTS-00121', 'identifier': [{'system'...","[{'id': 'KiTS-00121__c4kc_kits', 'identifier':...",[]
96,c43b0ae1-df35-4044-94aa-4785ea8d22a2,"[{'system': 'IDC', 'value': 'c43b0ae1-df35-404...",tasets-idc/c43b0ae1-df35-4044-94aa-4785ea8d22a...,Imaging,None,DICOM,c4kc_kits,drs://dg.4DFC:c43b0ae1-df35-4044-94aa-4785ea8d...,None,None,Imaging,CT,None,"[{'id': 'KiTS-00124', 'identifier': [{'system'...","[{'id': 'KiTS-00124__c4kc_kits', 'identifier':...",[]
97,e3b66d91-396f-4f89-b39d-e4e9dbbe2179,"[{'system': 'IDC', 'value': 'e3b66d91-396f-4f8...",tasets-idc/e3b66d91-396f-4f89-b39d-e4e9dbbe217...,Imaging,None,DICOM,c4kc_kits,drs://dg.4DFC:e3b66d91-396f-4f89-b39d-e4e9dbbe...,None,None,Imaging,CT,None,"[{'id': 'KiTS-00126', 'identifier': [{'system'...","[{'id': 'KiTS-00126__c4kc_kits', 'identifier':...",[]
98,0af13c34-7487-47e0-b3d6-a1c94ef0d193,"[{'system': 'IDC', 'value': '0af13c34-7487-47e...",tasets-idc/0af13c34-7487-47e0-b3d6-a1c94ef0d19...,Imaging,None,DICOM,c4kc_kits,drs://dg.4DFC:0af13c34-7487-47e0-b3d6-a1c94ef0...,None,None,Imaging,CT,None,"[{'id': 'KiTS-00132', 'identifier': [{'system'...","[{'id': 'KiTS-00132__c4kc_kits', 'identifier':...",[]


#### We can further subset by chaining queries

There are lots of Subjects in our general search for Kidney, so lets try to filter it to just cancer, we're most interested in early stage cancer, so we'll filter by Diagnosis.stage:


In [8]:
cancerquery = Q('ResearchSubject.Diagnosis.stage = "Stage I"').Or(Q('ResearchSubject.Diagnosis.stage = "Stage II"'))
kidneycancerquery = cancerquery.And(myquery)
kidneycancerdemographic = kidneycancerquery.run(version="all_Files_v3_0_w_RS")
kidneycancerdemographic

Getting results from database

Total execution time: 4036 ms



            QueryID: a20271d9-3120-467d-9cbc-0f4be65565c5
            Query:SELECT all_Files_v3_0_w_RS.* FROM gdc-bq-sample.dev.all_Files_v3_0_w_RS AS all_Files_v3_0_w_RS, UNNEST(all_Files_v3_0_w_RS.ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _ResearchSubject_Diagnosis WHERE (((UPPER(_ResearchSubject_Diagnosis.stage) = UPPER('Stage I')) OR (UPPER(_ResearchSubject_Diagnosis.stage) = UPPER('Stage II'))) AND (UPPER(_ResearchSubject.primary_diagnosis_site) = UPPER('Kidney')))
            Offset: 0
            Count: 100
            Total Row Count: 9982
            More pages: True
            

This is a much more managable number of subjects, and now they're targeted by our actual question. Let's peek at the data:

In [9]:
kidneycancerdemographic.to_dataframe().head()

,id,identifier,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number,Subject,ResearchSubject,Specimen
0,836e7306-19cc-11e9-99db-005056921935,"[{'system': 'PDC', 'value': '836e7306-19cc-11e...",07CPTAC_CCRCC_W_JHU_20171127_LUMOS_f22.raw,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC3-Discovery,drs://dg.4DFC:836e7306-19cc-11e9-99db-00505692...,753651146,102264f0bec47aa75ddd826a924369ac,Proteomic,None,None,"[{'id': 'QC3', 'identifier': [{'system': 'PDC'...",[{'id': '0b43d28c-1fba-11e9-b7f8-0a80fada099c'...,[{'id': '0635b755-204e-11e9-b7f8-0a80fada099c'...
1,836e7306-19cc-11e9-99db-005056921935,"[{'system': 'PDC', 'value': '836e7306-19cc-11e...",07CPTAC_CCRCC_W_JHU_20171127_LUMOS_f22.raw,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC3-Discovery,drs://dg.4DFC:836e7306-19cc-11e9-99db-00505692...,753651146,102264f0bec47aa75ddd826a924369ac,Proteomic,None,None,"[{'id': 'QC3', 'identifier': [{'system': 'PDC'...",[{'id': '0b43d28c-1fba-11e9-b7f8-0a80fada099c'...,[{'id': '0635b755-204e-11e9-b7f8-0a80fada099c'...
2,aef47e4c-1a00-11e9-b898-005056921935,"[{'system': 'PDC', 'value': 'aef47e4c-1a00-11e...",07CPTAC_CCRCC_P_JHU_20171212_LUMOS_f03.raw,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC3-Discovery,drs://dg.4DFC:aef47e4c-1a00-11e9-b898-00505692...,908045374,fd60952b13702194cc1db527adbc95a9,Proteomic,None,None,"[{'id': 'QC3', 'identifier': [{'system': 'PDC'...",[{'id': '0b43d28c-1fba-11e9-b7f8-0a80fada099c'...,[{'id': '0635b755-204e-11e9-b7f8-0a80fada099c'...
3,aef47e4c-1a00-11e9-b898-005056921935,"[{'system': 'PDC', 'value': 'aef47e4c-1a00-11e...",07CPTAC_CCRCC_P_JHU_20171212_LUMOS_f03.raw,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC3-Discovery,drs://dg.4DFC:aef47e4c-1a00-11e9-b898-00505692...,908045374,fd60952b13702194cc1db527adbc95a9,Proteomic,None,None,"[{'id': 'QC3', 'identifier': [{'system': 'PDC'...",[{'id': '0b43d28c-1fba-11e9-b7f8-0a80fada099c'...,[{'id': '0635b755-204e-11e9-b7f8-0a80fada099c'...
4,d0159292-1b58-11e9-b01e-005056921935,"[{'system': 'PDC', 'value': 'd0159292-1b58-11e...",07CPTAC_CCRCC_P_JHU_20171212_LUMOS_f07.mzid.gz,Peptide Spectral Matches,Open Standard,mzIdentML,CPTAC3-Discovery,drs://dg.4DFC:d0159292-1b58-11e9-b01e-00505692...,3405616,98c7e21ed95fa7d3964f82cca1a02d50,Proteomic,None,None,"[{'id': 'QC3', 'identifier': [{'system': 'PDC'...",[{'id': '0b43d28c-1fba-11e9-b7f8-0a80fada099c'...,[{'id': '0635b755-204e-11e9-b7f8-0a80fada099c'...


### File summary
Now that we know there are research subjects that meet our criteria, lets see what data exists about them. We can get a summary of the data for these subjects using the `counts()` feature:

In [10]:
kidneycancerquery.counts(version="all_Files_v3_0_w_RS").to_dataframe()#.iloc[0:3,0:2]

,system,subject_count,subject_files_count,researchsubject_count,researchsubject_files_count,specimen_count,specimen_files_count
0,IDC,106,0,0,0,0,0
1,PDC,119,3446,65,3446,380,3407
2,GDC,106,0,0,0,0,0


### What do these numbers mean?

---
    
- **system:** Which data source contributed this data? The CDA currently has data from IDC, PDC and GDC
- **subject_count:** How many unique individuals meet our query. Note that *within* a data source the number is of *unique* individuals, but the same individuals can have data at multiple centers. Here, there are 371 unique people in the IDC data, however up to 57 of those may be exactly the same people as are in the PDC data.
- **subject_files_count:** This tells you roughly how much data is available. It is the total count of files for all the subjects in `subject_count`, which is also the total number of files that match your search.
- **researchsubject_count:** Some data sources have individual subjects that are in multiple studies, when this happens the individual will have both a "subject" identifier and a "researchsubject" identifier. This column counts the latter. Zero in this column can mean either "there are no research_subjects that meet your search criteria" or "the data source for this row does not create special identifiers for subjects in multiple studies"
- **researchsubject_files_count:** This is the total count of files specific to researchsubjects in `researchsubject_count`. It is a subset of `subject_count`
- **specimen_count:** Some data sources track whether files come from specific specimens from a given individual. This column counts the number of specimens that meet your search criteria. Zero in this column can mean either "there are no specimens that meet your search criteria" or "the data source for this row does not track specimens seperately from subjects"
- **specimen_files_count:** This is the total count of files specific to specimens in `specimen_count`. It is a subset of both `subject_count` and `researchsubject_files_count`

---

## Retrieving data


Let's run the same query, but instead of asking for summary information, lets get the data about each file. We start this process the same way, by making a `Q` statement. We can reuse `cancerquery` here as well, and just run the `.files()` function on it: 

In [11]:
kidneycancerfiles = kidneycancerquery.files(version="all_Files_v3_0_w_RS")
kidneycancerfiles

Getting results from database




            QueryID: fd6fea46-b25a-42b8-92d3-563e93df8d42
            Query:SELECT all_Files_v3_0_w_RS.* FROM gdc-bq-sample.dev.all_Files_v3_0_w_RS AS all_Files_v3_0_w_RS, UNNEST(all_Files_v3_0_w_RS.ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _ResearchSubject_Diagnosis WHERE (((UPPER(_ResearchSubject_Diagnosis.stage) = UPPER('Stage I')) OR (UPPER(_ResearchSubject_Diagnosis.stage) = UPPER('Stage II'))) AND (UPPER(_ResearchSubject.primary_diagnosis_site) = UPPER('Kidney')))
            Offset: 0
            Count: 100
            Total Row Count: 9982
            More pages: True
            

In [12]:
kidneycancerfiles.to_dataframe()

,id,identifier,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number,Subject,ResearchSubject,Specimen
0,836e7306-19cc-11e9-99db-005056921935,"[{'system': 'PDC', 'value': '836e7306-19cc-11e...",07CPTAC_CCRCC_W_JHU_20171127_LUMOS_f22.raw,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC3-Discovery,drs://dg.4DFC:836e7306-19cc-11e9-99db-00505692...,753651146,102264f0bec47aa75ddd826a924369ac,Proteomic,None,None,"[{'id': 'QC3', 'identifier': [{'system': 'PDC'...",[{'id': '0b43d28c-1fba-11e9-b7f8-0a80fada099c'...,[{'id': '0635b755-204e-11e9-b7f8-0a80fada099c'...
1,836e7306-19cc-11e9-99db-005056921935,"[{'system': 'PDC', 'value': '836e7306-19cc-11e...",07CPTAC_CCRCC_W_JHU_20171127_LUMOS_f22.raw,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC3-Discovery,drs://dg.4DFC:836e7306-19cc-11e9-99db-00505692...,753651146,102264f0bec47aa75ddd826a924369ac,Proteomic,None,None,"[{'id': 'QC3', 'identifier': [{'system': 'PDC'...",[{'id': '0b43d28c-1fba-11e9-b7f8-0a80fada099c'...,[{'id': '0635b755-204e-11e9-b7f8-0a80fada099c'...
2,aef47e4c-1a00-11e9-b898-005056921935,"[{'system': 'PDC', 'value': 'aef47e4c-1a00-11e...",07CPTAC_CCRCC_P_JHU_20171212_LUMOS_f03.raw,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC3-Discovery,drs://dg.4DFC:aef47e4c-1a00-11e9-b898-00505692...,908045374,fd60952b13702194cc1db527adbc95a9,Proteomic,None,None,"[{'id': 'QC3', 'identifier': [{'system': 'PDC'...",[{'id': '0b43d28c-1fba-11e9-b7f8-0a80fada099c'...,[{'id': '0635b755-204e-11e9-b7f8-0a80fada099c'...
3,aef47e4c-1a00-11e9-b898-005056921935,"[{'system': 'PDC', 'value': 'aef47e4c-1a00-11e...",07CPTAC_CCRCC_P_JHU_20171212_LUMOS_f03.raw,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC3-Discovery,drs://dg.4DFC:aef47e4c-1a00-11e9-b898-00505692...,908045374,fd60952b13702194cc1db527adbc95a9,Proteomic,None,None,"[{'id': 'QC3', 'identifier': [{'system': 'PDC'...",[{'id': '0b43d28c-1fba-11e9-b7f8-0a80fada099c'...,[{'id': '0635b755-204e-11e9-b7f8-0a80fada099c'...
4,d0159292-1b58-11e9-b01e-005056921935,"[{'system': 'PDC', 'value': 'd0159292-1b58-11e...",07CPTAC_CCRCC_P_JHU_20171212_LUMOS_f07.mzid.gz,Peptide Spectral Matches,Open Standard,mzIdentML,CPTAC3-Discovery,drs://dg.4DFC:d0159292-1b58-11e9-b01e-00505692...,3405616,98c7e21ed95fa7d3964f82cca1a02d50,Proteomic,None,None,"[{'id': 'QC3', 'identifier': [{'system': 'PDC'...",[{'id': '0b43d28c-1fba-11e9-b7f8-0a80fada099c'...,[{'id': '0635b755-204e-11e9-b7f8-0a80fada099c'...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bd1df6ca-1b53-11e9-a7d8-005056921935,"[{'system': 'PDC', 'value': 'bd1df6ca-1b53-11e...",02CPTAC_CCRCC_W_JHU_20171003_LUMOS_f15.psm,Peptide Spectral Matches,Text,tsv,CPTAC3-Discovery,drs://dg.4DFC:bd1df6ca-1b53-11e9-a7d8-00505692...,3890682,aaf08b7d54c2920821c75acf681bf74c,Proteomic,None,None,"[{'id': 'C3N-01214', 'identifier': [{'system':...",[{'id': 'a767e303-1fb9-11e9-b7f8-0a80fada099c'...,[{'id': 'e2be63e9-204c-11e9-b7f8-0a80fada099c'...
96,055f62fe-1b4d-11e9-8e2e-005056921935,"[{'system': 'PDC', 'value': '055f62fe-1b4d-11e...",06CPTAC_CCRCC_P_JHU_20171124_LUMOS_f02.mzML.gz,Processed Mass Spectra,Open Standard,mzML,CPTAC3-Discovery,drs://dg.4DFC:055f62fe-1b4d-11e9-8e2e-00505692...,269075153,054afc6635be21e96d4bc33ebca802ea,Proteomic,None,None,"[{'id': 'QC2', 'identifier': [{'system': 'PDC'...",[{'id': '09a53ad8-1fba-11e9-b7f8-0a80fada099c'...,[{'id': '04afc8fb-204e-11e9-b7f8-0a80fada099c'...
97,055f62fe-1b4d-11e9-8e2e-005056921935,"[{'system': 'PDC', 'value': '055f62fe-1b4d-11e...",06CPTAC_CCRCC_P_JHU_20171124_LUMOS_f02.mzML.gz,Processed Mass Spectra,Open Standard,mzML,CPTAC3-Discovery,drs://dg.4DFC:055f62fe-1b4d-11e9-8e2e-00505692...,269075153,054afc6635be21e96d4bc33ebca802ea,Proteomic,None,None,"[{'id': 'QC2', 'identifier': [{'system': 'PDC'...",[{'id': '09a53ad8-1fba-11e9-b7f8-0a80fada099c'...,[{'id': '04afc8fb-204e-11e9-b7f8-0a80fada099c'...
98,0c844200-1b54-11e9-a7d8-005056921935,"[{'system': 'PD

What are all these fields?

---

- **id:** The unique identifier for this file
- **identifier:** An embedded array of information that includes the originating data center and the ID the file had there
- **label:** The full name of the file
- **data_catagory:** A desecription of the kind of general kind data the file holds
- **data_type:** A more specific descripton of the data type
- **file_format:** The extension of the file
- **associated_project:** The name the data center uses for the study this file was generated for
- **drs_uri:** A unique identifier that can be used to retreive this specific file from a server
- **byte_size:** Size of the file in bytes
- **checksum:** The md5 value for the file
- **data_modality:** A high level descriptor of file data, always one of "Genomic", "Proteomic", or "Imaging"
- **imaging_modality** For files with the `data_modality` of "Imaging", a descriptor for the image type
- **dbgap_accession_number:** An identifier for the dbGaP project this file belongs to
- **Subject:** An embedded array of information that includes the originating data center and the Subject ID the file had there
- **ResearchSubject:** An embedded array of information that includes the originating data center and the ResearchSubject ID the file had there
- **Specimen:** An embedded array of information that includes the originating data center and the Specimen ID the file had there

---
